###Team Members = Tomiris, Kulash, Yussuf
***NN Model - LSTM***

## Import Library

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

import sys
sys.path.insert(0,'/content/drive/MyDrive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/ROBT513_BMI/BMI_final

/content/drive/MyDrive/ROBT513_BMI/BMI_final


In [3]:
!pip install mne==0.18.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 20.1 MB/s 
  Created wheel for mne: filename=mne-0.18.0-py3-none-any.whl size=6313880 sha256=56846ad8b47e7f9b44139357d8e82c2199bb7443827dfbc0b4f9c6a63a0712ce
  Stored in directory: /root/.cache/pip/wheels/e4/3c/6a/73096375347433a5e5978da35abdec5423a02e34d307924917
Successfully built mne


In [4]:
from nu_smrutils import *

import torch 
import itertools
import pandas as pd 
import pickle 
import numpy as np
import mne

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable 

%matplotlib inline
%load_ext autoreload
%autoreload 2
#%matplotlib qt 
torch.manual_seed(0)

## Necessary functions

In [5]:
scaler = StandardScaler()

def preprocessing(data):
  sh1=data.shape[1]
  sh2=data.shape[2]
  data=data.reshape(data.shape[0],-1)
  data=scaler.fit_transform(data)
  data=data.reshape(data.shape[0],sh1,sh2)
  return data

In [6]:
#by default 
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if dev.type == 'cuda':
   print('Your GPU device name :', torch.cuda.get_device_name())      

In [7]:
from torch.utils.data import TensorDataset, DataLoader

def get_data_loaders(dat, batch_size, EEGNET = None, dev=dev):    
    # convert data dimensions to into to gray scale image format
    #if EEGNET: ### EEGNet model requires the last dimension to be 1 
        #ff = lambda dat: torch.unsqueeze(dat, dim = -1)    
    #else:
        #ff = lambda dat: torch.unsqueeze(dat, dim = 1)    
        
    x_train, x_valid, x_test =  (dat['xtrain'],dat['xvalid'],dat['xtest'])    
    y_train, y_valid, y_test =  dat['ytrain'], dat['yvalid'], dat['ytest']
    print('Input data shape', x_train.shape)       
    
    # TensorDataset & Dataloader    
    train_dat = TensorDataset(x_train.to(dev), y_train.to(dev)) 
    val_dat   = TensorDataset(x_valid.to(dev), y_valid.to(dev)) 
    
    train_loader = DataLoader(train_dat, batch_size = batch_size, shuffle = True)
    val_loader   = DataLoader(val_dat,   batch_size = batch_size, shuffle = False)

    output = dict(dset_loaders = {'train': train_loader, 'val': val_loader}, 
                  dset_sizes  =  {'train': len(x_train), 'val': len(x_valid)},
                  test_data   =  {'x_test' : x_test, 'y_test' : y_test})          
    return output

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim = 100, layer_dim = 1, output_dim = 2):
        super(LSTMModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(out[:, -1, :]) 

        return out

## Dataset BNCI2014004


Select Dataset

In [9]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014004'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [10]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=torch.tensor(preprocessing(datt[i]['xtrain'])).type('torch.FloatTensor')
  datt[i]['xvalid']=torch.tensor(preprocessing(datt[i]['xvalid'])).type('torch.FloatTensor')
  datt[i]['xtest']=torch.tensor(preprocessing(datt[i]['xtest'])).type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([982, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1068, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])
Shape after data augmentation : torch.Size([1098, 3, 321])
Shape after data augmentation : torch.Size([1040, 3, 321])

Cropping
torch.Size([3120, 3, 120])
torch.Size([2946, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3204, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3120, 3, 120])
torch.Size([3294, 3, 120])
tor

Training loop for dataset 

In [11]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = LSTMModel(input_size[2])

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0109, acc: 0.5010
val loss: 0.0126, acc: 0.5000
Epoch 2/150
train loss: 0.0109, acc: 0.5167
val loss: 0.0126, acc: 0.4891
Epoch 3/150
train loss: 0.0108, acc: 0.5266
val loss: 0.0126, acc: 0.4928
Epoch 4/150
train loss: 0.0108, acc: 0.5429
val loss: 0.0126, acc: 0.5036
Epoch 5/150
train loss: 0.0107, acc: 0.5660
val loss: 0.0126, acc: 0.5290
Epoch 6/150
train loss: 0.0107, acc: 0.5776
val loss: 0.0126, acc: 0.5326
Epoch 7/150
train loss: 0.0107, acc: 0.5981
val loss: 0.0126, acc: 0.5362
Epoch 8/150
train loss: 0.0106, acc: 0.6074
val loss: 0.0126, acc: 0.5326
Epoch 9/150
train loss: 0.0106, acc: 0.6196
val loss: 0.0126, acc: 0.5435
Epoch 10/150
train loss: 0.0105, acc: 0.6285
val loss: 0.0126, acc: 0.5362
Epoch 11/150
train loss: 0.0104, acc: 0.6301
val loss: 0.0126, acc: 0.5399
Epoch 12/150
train loss: 0.0104, acc: 0.6369
val loss: 0.0127, acc: 0.5326
Epoch 13/150
train loss: 0.0103, acc: 0.6446
val loss: 0.0127,

In [12]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   0.834615  0.634058      0.62    87
Subject2   0.522403  0.532567      0.46     4
Subject3   0.541346  0.539855      0.50     3
Subject4   0.884207  0.807018      0.84    41
Subject5   0.635456  0.585965      0.48    13
Subject6   0.914744  0.572464      0.54   134
Subject7   0.821795  0.572464      0.74   100
Subject8   0.902550  0.766323      0.74    56
Subject9   0.879487  0.583333      0.64   143

Average between subjects: 0.62
Max test_score between subjects: 0.84


## Dataset BNCI2014001


Select Dataset

In [14]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'BNCI2014001'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 9


Load subject specific data and preprocess it

In [15]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=torch.tensor(preprocessing(datt[i]['xtrain'])).type('torch.FloatTensor')
  datt[i]['xvalid']=torch.tensor(preprocessing(datt[i]['xvalid'])).type('torch.FloatTensor')
  datt[i]['xtest']=torch.tensor(preprocessing(datt[i]['xtest'])).type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
list index out of range

Augment
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])
Shape after data augmentation : torch.Size([414, 22, 321])

Cropping
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 120])
torch.Size([1242, 22, 

Training loop for dataset 

In [16]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = LSTMModel(input_size[2])

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0111, acc: 0.5233
val loss: 0.0125, acc: 0.5315
Epoch 2/150
train loss: 0.0110, acc: 0.5451
val loss: 0.0125, acc: 0.5045
Epoch 3/150
train loss: 0.0110, acc: 0.5660
val loss: 0.0125, acc: 0.4775
Epoch 4/150
train loss: 0.0109, acc: 0.5837
val loss: 0.0125, acc: 0.4955
Epoch 5/150
train loss: 0.0109, acc: 0.6023
val loss: 0.0126, acc: 0.5045
Epoch 6/150
train loss: 0.0108, acc: 0.6047
val loss: 0.0126, acc: 0.4955
Epoch 7/150
train loss: 0.0108, acc: 0.6224
val loss: 0.0126, acc: 0.5135
Epoch 8/150
train loss: 0.0107, acc: 0.6304
val loss: 0.0126, acc: 0.5135
Epoch 9/150
train loss: 0.0106, acc: 0.6320
val loss: 0.0127, acc: 0.5225
Epoch 10/150
train loss: 0.0106, acc: 0.6401
val loss: 0.0128, acc: 0.5315
Epoch 11/150
train loss: 0.0105, acc: 0.6514
val loss: 0.0128, acc: 0.5315
Epoch 12/150
train loss: 0.0105, acc: 0.6570
val loss: 0.0129, acc: 0.5225
Epoch 13/150
train loss: 0.0104, acc: 0.6618
val loss: 0.0130,

In [17]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

          Train_Acc   Val_Acc  Test_Acc Epoch
Subject1   0.670692  0.540541      0.54    14
Subject2   0.994364  0.549550      0.56    66
Subject3   0.916264  0.558559      0.52    43
Subject4   0.679549  0.549550      0.52    12
Subject5   0.938808  0.504505      0.50    54
Subject6   0.692432  0.549550      0.66    13
Subject7   0.545089  0.549550      0.54     3
Subject8   0.935588  0.513514      0.52    39
Subject9   0.981481  0.666667      0.50    47

Average between subjects: 0.54
Max test_score between subjects: 0.66


## Dataset Weibo2014


Select Dataset

In [18]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Weibo2014'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 10


Load subject specific data and preprocess it

In [19]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=torch.tensor(preprocessing(datt[i]['xtrain'])).type('torch.FloatTensor')
  datt[i]['xvalid']=torch.tensor(preprocessing(datt[i]['xvalid'])).type('torch.FloatTensor')
  datt[i]['xtest']=torch.tensor(preprocessing(datt[i]['xtest'])).type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter


/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
list index out of range

Augment
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([202, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])
Shape after data augmentation : torch.Size([230, 60, 321])

Cropping
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torch.Size([690, 60, 120])
torc

Training loop for dataset 

In [20]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = LSTMModel(input_size[2])

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

----------------Subject 1---------------------
Epoch 1/150
train loss: 0.0111, acc: 0.5014
val loss: 0.0112, acc: 0.4762
Epoch 2/150
train loss: 0.0111, acc: 0.5130
val loss: 0.0112, acc: 0.4921
Epoch 3/150
train loss: 0.0110, acc: 0.5290
val loss: 0.0112, acc: 0.5079
Epoch 4/150
train loss: 0.0109, acc: 0.5449
val loss: 0.0112, acc: 0.5238
Epoch 5/150
train loss: 0.0109, acc: 0.5609
val loss: 0.0112, acc: 0.5397
Epoch 6/150
train loss: 0.0108, acc: 0.5739
val loss: 0.0112, acc: 0.4921
Epoch 7/150
train loss: 0.0107, acc: 0.6000
val loss: 0.0112, acc: 0.4921
Epoch 8/150
train loss: 0.0107, acc: 0.6188
val loss: 0.0112, acc: 0.5397
Epoch 9/150
train loss: 0.0106, acc: 0.6377
val loss: 0.0113, acc: 0.5397
Epoch 10/150
train loss: 0.0105, acc: 0.6449
val loss: 0.0113, acc: 0.5397
Epoch 11/150
train loss: 0.0105, acc: 0.6652
val loss: 0.0113, acc: 0.5556
Epoch 12/150
train loss: 0.0104, acc: 0.6783
val loss: 0.0113, acc: 0.5556
Epoch 13/150
train loss: 0.0103, acc: 0.6870
val loss: 0.0114,

In [21]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

           Train_Acc   Val_Acc  Test_Acc Epoch
Subject1    0.856522  0.571429      0.44    35
Subject2    0.992754  0.682540      0.44    91
Subject3    0.997101  0.634921      0.54   131
Subject4    0.975362  0.587302      0.50    70
Subject5    0.804348  0.634921      0.54    29
Subject6    0.660066  0.574074      0.44    10
Subject7    0.940580  0.682540      0.46    51
Subject8    0.755072  0.603175      0.58    25
Subject9    0.859420  0.682540      0.50    39
Subject10   0.553623  0.571429      0.56     4

Average between subjects: 0.50
Max test_score between subjects: 0.58


## Dataset Physionet


Select Dataset

In [22]:
dname = dict(BNCI2014004 = 'aBNCI2014004R.pickle',
             BNCI2014001 = 'aBNCI2014001R.pickle',
             Weibo2014   = 'aWeibo2014R.pickle',
             Physionet   = 'aPhysionetRR.pickle')

itemname = 'Physionet'

filename = dname[itemname]
iname = itemname + '__'    

data = loaddat(filename)
print('Number of subjects in data :', len(data))

Number of subjects in data : 109


Load subject specific data and preprocess it

In [23]:
print('\nfilter')
for i in range(len(data)):
  data[i].filter(l_freq = 8, h_freq = 12)

#--------------------------------------------------------------------------

subjectIndex = list(range(108))
class_name = ['left_hand', 'right_hand']
print('\nsubject_specific')
datt = subject_specific(data, subjectIndex, class_name, test_size = 0.15)


#--------------------------------------------------------------------------
# augment only the training data 
print('\nAugment')
augmdata = dict(std_dev = 0.00001,  multiple = 2)

for ii, dat in enumerate(datt):    
    xtrain, ytrain = augment_dataset(dat['xtrain'], dat['ytrain'], 
                                     std_dev = augmdata['std_dev'], 
                                     multiple = augmdata['multiple'])
    
    print("Shape after data augmentation :", xtrain.shape)
    datt[ii]['xtrain'], datt[ii]['ytrain'] = xtrain, ytrain

#--------------------------------------------------------------------------
# Data Cropping
print('\nCropping')
fs = 80 # sampling frequency 
crop_len = 1 #or None
crop = dict(fs = fs, crop_len = crop_len)

for ii, dat in enumerate(datt):
    if crop['crop_len']:
       X_train, y_train = crop_data(crop['fs'], crop['crop_len'], dat['xtrain'], 
                                    dat['ytrain'],xpercent = 50)
       X_valid, y_valid = crop_data(crop['fs'], crop['crop_len'], dat['xvalid'], 
                                    dat['yvalid'],xpercent = 50)
       X_test,  y_test  = crop_data(crop['fs'], crop['crop_len'], dat['xtest'], 
                                    dat['ytest'], xpercent = 50)
       
       print(X_train.shape)
       datt[ii] = dict(xtrain = X_train, xvalid = X_valid, xtest = X_test,
                       ytrain = y_train, yvalid = y_valid, ytest = y_test) 
                        
#--------------------------------------------------------------------------
# Preprocessing dataset
print('\nPreprocessing')
for i in range(0,len(datt)):
  datt[i]['xtrain']=torch.tensor(preprocessing(datt[i]['xtrain'])).type('torch.FloatTensor')
  datt[i]['xvalid']=torch.tensor(preprocessing(datt[i]['xvalid'])).type('torch.FloatTensor')
  datt[i]['xtest']=torch.tensor(preprocessing(datt[i]['xtest'])).type('torch.FloatTensor')    

#--------------------------------------------------------------------------
# Dataloaders
print('\nDataloaders')
for ii, dat in enumerate(datt):  
    print('Subjects :', ii+1)
    datt[ii] = get_data_loaders(dat, batch_size = 64)


filter
Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



...

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec

Setting up band-pass filter from 8 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 133 samples (1.663 sec)



/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/io/pick.py:360: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pick = np.zeros(nchan, dtype=np.bool)
/usr/local/lib/python3.8/dist-packages/mne/filter.py:182: Deprec


subject_specific
Loading subject: 0
Loading subject: 1
Loading subject: 2
Loading subject: 3
Loading subject: 4
Loading subject: 5
Loading subject: 6
Loading subject: 7
Loading subject: 8
Loading subject: 9
Loading subject: 10
Loading subject: 11
Loading subject: 12
Loading subject: 13
Loading subject: 14
Loading subject: 15
Loading subject: 16
Loading subject: 17
Loading subject: 18
Loading subject: 19
Loading subject: 20
Loading subject: 21
Loading subject: 22
Loading subject: 23
Loading subject: 24
Loading subject: 25
Loading subject: 26
Loading subject: 27
Loading subject: 28
Loading subject: 29
Loading subject: 30
Loading subject: 31
Loading subject: 32
Loading subject: 33
Loading subject: 34
Loading subject: 35
Loading subject: 36
Loading subject: 37
Loading subject: 38
Loading subject: 39
Loading subject: 40
Loading subject: 41
Loading subject: 42
Loading subject: 43
Loading subject: 44
Loading subject: 45
Loading subject: 46
Loading subject: 47
Loading subject: 48
Loading subj

Training loop for dataset 

In [24]:
#Hyperparameter settings
num_epochs = 150
learning_rate = 1e-4
weight_decay = 1e-4  
batch_size = 64
verbose = 2

fs = 80
# define kernel size in terms of ms length 
time_window = 100 #ms
width = time_window*fs//1000    

# ker = 8 #timelength//chans 
h, w = 3, 1  #hight and width of a rectangular kernel    

#% used to save the results table 
results = {}        
table = pd.DataFrame(columns = ['Train_Acc', 'Val_Acc', 'Test_Acc', 'Epoch']) 
sum=0
num=0
max=0

for i in range(len(datt)): 
  subject = i
  print('----------------Subject '+str(subject+1) + '---------------------')
  description = 'Subject'+str(subject+1)
  dat = datt[subject]

  #% get input size (channel x timepoints)
  input_size = (1, dat['test_data']['x_test'].shape[-2], 
                 dat['test_data']['x_test'].shape[-1])

  model = LSTMModel(input_size[2])

  # optimizer and the loss function definition 
  optimizer = torch.optim.Adam(model.parameters(), 
                             lr = learning_rate,
                             weight_decay = weight_decay)
  criterion = torch.nn.CrossEntropyLoss()

  model.to(dev)  
  criterion.to(dev)       

  #print("Model architecture >>>", model)

  #******** Training loop *********    
  best_model, train_losses, val_losses, train_accs, val_accs, info =\
      train_model(model, dat['dset_loaders'], dat['dset_sizes'], 
                  criterion, optimizer, dev=dev, lr_scheduler=False, 
                  num_epochs = num_epochs, verbose = verbose)
          
  
  test_samples = 50
  x_test = dat['test_data']['x_test'][:test_samples,:,:] 
  y_test = dat['test_data']['y_test'][:test_samples] 
  #print(x_test.shape)

  # predict test data 
  preds = best_model(x_test.to(dev)) 
  preds_class = preds.data.max(1)[1]

  # get the accuracy 
  corrects = torch.sum(preds_class == y_test.data.to(dev))     
  test_acc = corrects.cpu().numpy()/x_test.shape[0]
  print("Test Accuracy :", test_acc) 
  sum+=test_acc
  num+=1
  if test_acc>max:
    max=test_acc
  # save results       
  tab = dict(Train_Acc= train_accs[info['best_epoch']],
            Val_Acc  = val_accs[info['best_epoch']],   
            Test_Acc = test_acc, Epoch = info['best_epoch'] + 1)         

  table.loc[description] = tab  
  val_acc = np.max(val_accs)
  print('\n---------------------------------------------------------------------\n')

Streaming output truncated to the last 5000 lines.
train loss: 0.0111, acc: 0.5776
val loss: 0.0639, acc: 0.4167
Epoch 14/150
train loss: 0.0111, acc: 0.5862
val loss: 0.0640, acc: 0.4167
Epoch 15/150
train loss: 0.0110, acc: 0.5948
val loss: 0.0641, acc: 0.4167
Epoch 16/150
train loss: 0.0109, acc: 0.6034
val loss: 0.0642, acc: 0.4167
Epoch 17/150
train loss: 0.0109, acc: 0.6121
val loss: 0.0642, acc: 0.4167
Epoch 18/150
train loss: 0.0108, acc: 0.6293
val loss: 0.0643, acc: 0.4167
Epoch 19/150
train loss: 0.0107, acc: 0.6379
val loss: 0.0644, acc: 0.4167
Epoch 20/150
train loss: 0.0107, acc: 0.6466
val loss: 0.0645, acc: 0.4167
Epoch 21/150
train loss: 0.0106, acc: 0.6638
val loss: 0.0645, acc: 0.4167
Epoch 22/150
train loss: 0.0106, acc: 0.6724
val loss: 0.0646, acc: 0.4167
Epoch 23/150
train loss: 0.0105, acc: 0.6983
val loss: 0.0647, acc: 0.4167
Epoch 24/150
train loss: 0.0105, acc: 0.7155
val loss: 0.0649, acc: 0.4167
Epoch 25/150
train loss: 0.0104, acc: 0.7500
val loss: 0.0650,

In [25]:
print(table)
results[description] = dict(train_accs = train_accs, val_accs =  val_accs,                                
                            ytrain = info['ytrain'], yval= info['yval'])      

fname = iname + 'CNN_POOLED' + description + '_' + str(val_acc)[:4]
torch.save(best_model.state_dict(), fname) 

print(f"\nAverage between subjects: {(sum/num):.2f}")
print(f"Max test_score between subjects: {max:.2f}")

            Train_Acc   Val_Acc  Test_Acc Epoch
Subject1     0.671875  0.416667  0.642857     4
Subject2     0.948276  0.666667  0.500000    88
Subject3     1.000000  0.750000  0.428571    97
Subject4     1.000000  0.833333  0.571429   106
Subject5     0.543103  0.333333  0.571429     1
...               ...       ...       ...   ...
Subject104   0.500000  0.833333  0.416667     1
Subject105   0.543103  0.416667  0.571429     5
Subject106   0.742188  0.583333  0.214286    23
Subject107   1.000000  0.416667  0.571429   114
Subject108   0.422414  0.500000  0.357143     1

[108 rows x 4 columns]

Average between subjects: 0.49
Max test_score between subjects: 0.79


In [37]:
table[:]

,Train_Acc,Val_Acc,Test_Acc,Epoch
Subject1,0.671875,0.416667,0.642857,4
Subject2,0.948276,0.666667,0.500000,88
Subject3,1.000000,0.750000,0.428571,97
Subject4,1.000000,0.833333,0.571429,106
Subject5,0.543103,0.333333,0.571429,1
...,...,...,...,...
Subject104,0.500000,0.833333,0.416667,1
Subject105,0.543103,0.416667,0.571429,5
Subject106,0.742188,0.583333,0.214286,23
Subject107,1.000000,0.416667,0.571429,114
